In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
titanic = pd.read_csv(r"/kaggle/input/titanic/train.csv")

In [3]:
titanic = titanic.drop(['Name','Ticket'],axis=1)
y = titanic["Survived"]

In [4]:
feat = ['Pclass','Sex','Fare','Embarked','SibSp','Parch']
x = pd.get_dummies(titanic[feat])

In [5]:
x.head()

,Pclass,Fare,SibSp,Parch,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,7.2500,1,0,False,True,False,False,True
1,1,71.2833,1,0,True,False,True,False,False
2,3,7.9250,0,0,True,False,False,False,True
3,1,53.1000,1,0,True,False,False,False,True
4,3,8.0500,0,0,False,True,False,False,True


In [6]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.25,stratify=x[["Sex_male","Sex_female","Pclass"]],random_state=45)

In [7]:
lregr = LogisticRegression(max_iter=2000)
lregr.fit(x,y)

LogisticRegression(max_iter=2000)

In [8]:
test = pd.read_csv("/kaggle/input/titanic/test.csv")
test = pd.get_dummies(test[feat])

In [9]:
mean_fare_by_pclass = test.groupby("Pclass")["Fare"].transform("mean")

# Use the calculated means to fill missing values in the Fare column
test["Fare"].fillna(mean_fare_by_pclass, inplace=True)

In [10]:
test.isna().sum()

Pclass        0
Fare          0
SibSp         0
Parch         0
Sex_female    0
Sex_male      0
Embarked_C    0
Embarked_Q    0
Embarked_S    0
dtype: int64

In [11]:
preds = lregr.predict(test)

In [12]:
preds.shape

(418,)

In [13]:
x = pd.read_csv(r"/kaggle/input/titanic/test.csv")
x.shape

(418, 11)

In [14]:
new_df = pd.DataFrame({'PassengerId': x['PassengerId']})

# Add the 'Survived' column from the 'preds' variable
new_df['Survived'] = preds

# Display the first few rows of the new DataFrame
print(new_df.head())

   PassengerId  Survived
0          892         0
1          893         1
2          894         0
3          895         0
4          896         1


In [15]:
new_df.to_csv('/kaggle/working/submission8.csv', index=False)
